In [9]:
!wget "https://drive.google.com/file/d/1VnyY9qzZzHzXOx1_ngiEVqJlWtlWu9lo" -O dataset.csv

--2023-10-26 11:01:52--  https://drive.google.com/file/d/1VnyY9qzZzHzXOx1_ngiEVqJlWtlWu9lo
Resolving drive.google.com (drive.google.com)... 142.250.152.138, 142.250.152.139, 142.250.152.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.152.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/1VnyY9qzZzHzXOx1_ngiEVqJlWtlWu9lo/ [following]
--2023-10-26 11:01:53--  https://drive.google.com/file/d/1VnyY9qzZzHzXOx1_ngiEVqJlWtlWu9lo/
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/1VnyY9qzZzHzXOx1_ngiEVqJlWtlWu9lo/edit [following]
--2023-10-26 11:01:53--  https://drive.google.com/file/d/1VnyY9qzZzHzXOx1_ngiEVqJlWtlWu9lo/edit
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dataset.csv’

dataset.csv    

# Reading and Cleaning the Data


First read the CSV file in pandas and intialise index as column 0.

In [12]:
import pandas as pd
matches = pd.read_csv("matches.csv", index_col=0)

Convert the data in the date field into a datetime object as we are dealing with time series data.

In [14]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


# Creating Predictors

A Machine Learning model such as this needs numerical data within which it will try to learn relationships, to predict some future target. We need to select and/or create these predictors within our dataset.

In [ ]:
matches["date"] = pd.to_datetime(matches["date"])

In [ ]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [ ]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [ ]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [ ]:
matches["day_code"] = matches["date"].dt.dayofweek

Create a target column within the dataframe which will be the target of our prediction. This target value should be a 1 for a win and a 0 for a loss or draw.

In [ ]:
matches["target"] = (matches["result"] == "W").astype("int")

# Initializing and Training the Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=1000, min_samples_split=10, random_state=1)

In [ ]:
train = matches[matches["date"] < '2022-01-01']
test = matches[matches["date"] > '2022-01-01']

In [ ]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [ ]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=1000, random_state=1)

# Generating Predictions and Testing the Accuracy of the Model


In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc = accuracy_score(test["target"], preds)
print(f"Accuracy of the model is {acc*100}%")

Accuracy of the model is 60.86956521739131%
